# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 22 2023 12:54PM,263416,10,Metha-9505134,"Methapharm, Inc.",Released
1,May 22 2023 12:48PM,263415,10,CTF0011761,"Citieffe, Inc.",Released
2,May 22 2023 12:48PM,263415,10,CTF0011762,"Citieffe, Inc.",Released
3,May 22 2023 12:48PM,263415,10,CTF0011763,"Citieffe, Inc.",Released
4,May 22 2023 12:48PM,263415,10,CTF0011764,"Citieffe, Inc.",Released
5,May 22 2023 12:43PM,263414,19,ADV80019047,"AdvaGen Pharma, Ltd",Released
6,May 22 2023 12:43PM,263414,19,ADV80019048,"AdvaGen Pharma, Ltd",Released
7,May 22 2023 12:43PM,263414,19,ADV80019049,"AdvaGen Pharma, Ltd",Released
8,May 22 2023 12:43PM,263414,19,ADV80019050,"AdvaGen Pharma, Ltd",Released
9,May 22 2023 12:43PM,263414,19,ADV80019051,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
27,263406,Released,28
28,263413,Released,1
29,263414,Released,13
30,263415,Released,4
31,263416,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
263406,NaN,NaN,28.0
263413,NaN,NaN,1.0
263414,NaN,NaN,13.0
263415,NaN,NaN,4.0
263416,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
263367,0.0,67.0,0.0
263369,0.0,1.0,0.0
263370,0.0,0.0,1.0
263372,0.0,0.0,8.0
263373,0.0,0.0,12.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
263367,0,67,0
263369,0,1,0
263370,0,0,1
263372,0,0,8
263373,0,0,12


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,263367,0,67,0
1,263369,0,1,0
2,263370,0,0,1
3,263372,0,0,8
4,263373,0,0,12


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,263367,,67,
1,263369,,1,
2,263370,,,1
3,263372,,,8
4,263373,,,12


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 22 2023 12:54PM,263416,10,"Methapharm, Inc."
1,May 22 2023 12:48PM,263415,10,"Citieffe, Inc."
5,May 22 2023 12:43PM,263414,19,"AdvaGen Pharma, Ltd"
18,May 22 2023 12:43PM,263413,20,"AdvaGen Pharma, Ltd"
19,May 22 2023 12:24PM,263406,15,"Brookfield Pharmaceuticals, LLC"
47,May 22 2023 12:11PM,263405,16,Sartorius Bioprocess Solutions
48,May 22 2023 12:06PM,263404,10,"Methapharm, Inc."
53,May 22 2023 12:00PM,263402,19,"Avet Pharmaceuticals Labs, Inc"
54,May 22 2023 11:58AM,263401,10,"Nextsource Biotechnology, LLC"
55,May 22 2023 11:57AM,263400,10,Eye Pharma Inc


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 22 2023 12:54PM,263416,10,"Methapharm, Inc.",,,1
1,May 22 2023 12:48PM,263415,10,"Citieffe, Inc.",,,4
2,May 22 2023 12:43PM,263414,19,"AdvaGen Pharma, Ltd",,,13
3,May 22 2023 12:43PM,263413,20,"AdvaGen Pharma, Ltd",,,1
4,May 22 2023 12:24PM,263406,15,"Brookfield Pharmaceuticals, LLC",,,28
5,May 22 2023 12:11PM,263405,16,Sartorius Bioprocess Solutions,,,1
6,May 22 2023 12:06PM,263404,10,"Methapharm, Inc.",,,5
7,May 22 2023 12:00PM,263402,19,"Avet Pharmaceuticals Labs, Inc",,,1
8,May 22 2023 11:58AM,263401,10,"Nextsource Biotechnology, LLC",,,1
9,May 22 2023 11:57AM,263400,10,Eye Pharma Inc,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 22 2023 12:54PM,263416,10,"Methapharm, Inc.",1,,
1,May 22 2023 12:48PM,263415,10,"Citieffe, Inc.",4,,
2,May 22 2023 12:43PM,263414,19,"AdvaGen Pharma, Ltd",13,,
3,May 22 2023 12:43PM,263413,20,"AdvaGen Pharma, Ltd",1,,
4,May 22 2023 12:24PM,263406,15,"Brookfield Pharmaceuticals, LLC",28,,
5,May 22 2023 12:11PM,263405,16,Sartorius Bioprocess Solutions,1,,
6,May 22 2023 12:06PM,263404,10,"Methapharm, Inc.",5,,
7,May 22 2023 12:00PM,263402,19,"Avet Pharmaceuticals Labs, Inc",1,,
8,May 22 2023 11:58AM,263401,10,"Nextsource Biotechnology, LLC",1,,
9,May 22 2023 11:57AM,263400,10,Eye Pharma Inc,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 22 2023 12:54PM,263416,10,"Methapharm, Inc.",1,,
1,May 22 2023 12:48PM,263415,10,"Citieffe, Inc.",4,,
2,May 22 2023 12:43PM,263414,19,"AdvaGen Pharma, Ltd",13,,
3,May 22 2023 12:43PM,263413,20,"AdvaGen Pharma, Ltd",1,,
4,May 22 2023 12:24PM,263406,15,"Brookfield Pharmaceuticals, LLC",28,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 22 2023 12:54PM,263416,10,"Methapharm, Inc.",1.0,NaN,NaN
1,May 22 2023 12:48PM,263415,10,"Citieffe, Inc.",4.0,NaN,NaN
2,May 22 2023 12:43PM,263414,19,"AdvaGen Pharma, Ltd",13.0,NaN,NaN
3,May 22 2023 12:43PM,263413,20,"AdvaGen Pharma, Ltd",1.0,NaN,NaN
4,May 22 2023 12:24PM,263406,15,"Brookfield Pharmaceuticals, LLC",28.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 22 2023 12:54PM,263416,10,"Methapharm, Inc.",1.0,0.0,0.0
1,May 22 2023 12:48PM,263415,10,"Citieffe, Inc.",4.0,0.0,0.0
2,May 22 2023 12:43PM,263414,19,"AdvaGen Pharma, Ltd",13.0,0.0,0.0
3,May 22 2023 12:43PM,263413,20,"AdvaGen Pharma, Ltd",1.0,0.0,0.0
4,May 22 2023 12:24PM,263406,15,"Brookfield Pharmaceuticals, LLC",28.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3424068,54.0,67.0,0.0
15,1843728,106.0,40.0,3.0
16,263405,1.0,0.0,0.0
19,1316987,28.0,33.0,0.0
20,263413,1.0,0.0,0.0
21,263376,1.0,0.0,0.0
22,263369,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3424068,54.0,67.0,0.0
1,15,1843728,106.0,40.0,3.0
2,16,263405,1.0,0.0,0.0
3,19,1316987,28.0,33.0,0.0
4,20,263413,1.0,0.0,0.0
5,21,263376,1.0,0.0,0.0
6,22,263369,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,54.0,67.0,0.0
1,15,106.0,40.0,3.0
2,16,1.0,0.0,0.0
3,19,28.0,33.0,0.0
4,20,1.0,0.0,0.0
5,21,1.0,0.0,0.0
6,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,54.0
1,15,Released,106.0
2,16,Released,1.0
3,19,Released,28.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21,22
Status,,,,,,,
Completed,0.0,3.0,0.0,0.0,0.0,0.0,0.0
Executing,67.0,40.0,0.0,33.0,0.0,0.0,1.0
Released,54.0,106.0,1.0,28.0,1.0,1.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21,22
0,Completed,0.0,3.0,0.0,0.0,0.0,0.0,0.0
1,Executing,67.0,40.0,0.0,33.0,0.0,0.0,1.0
2,Released,54.0,106.0,1.0,28.0,1.0,1.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21,22
0,Completed,0.0,3.0,0.0,0.0,0.0,0.0,0.0
1,Executing,67.0,40.0,0.0,33.0,0.0,0.0,1.0
2,Released,54.0,106.0,1.0,28.0,1.0,1.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()